In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
from random import choice
import sys
from collections import deque

In [2]:
# Use Charlie Parker's page as root
BASE_URL = "http://www.allmusic.com/artist/charlie-parker-mn0000211758"

# Path to files
artist_file = "data/allmusic/artists.txt"
influence_file = "data/allmusic/influences.txt"

# List of headers to spoof
headers = ["Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0",
          "Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0",
          "Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16",
          "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
          ]

In [3]:
artist_out = open(artist_file, 'w')
influence_out = open(influence_file, 'w')
artist_writer = csv.writer(artist_out)
influence_writer = csv.writer(influence_out)

# Explored set
explored = set()

# Place root page onto the queue
url_queue = deque([BASE_URL])

# Run Breadth First Search on musician pages
while len(url_queue) != 0:
    print "Size of queue:", len(url_queue)
    print "Artists explored:", len(explored)
    
    current_base_url = url_queue.popleft()
    current_url = current_base_url + '/related'
    
    try:
        session = requests.Session()

        # Spoof a random header
        session.headers.update({'User-Agent': choice(headers)})
        page = session.get(current_url)

        # Sleep for random number of seconds
        sleep(choice([4,5]))
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract Name
        try:
            name = soup.find_all('h1', class_='artist-name')[0].text.strip()
        except:
            print "Name Extraction Failure", current_url
            name = "NA"

        # Extract Active Period
        try:
            active = soup.find_all('div', class_='active-dates')[0].find_all('div')[0].text
        except:
            print "Active Period Extraction Failure", current_url
            active = "NA"

        # Extract the genres associated with the musician
        try:
            genre_tags = soup.find_all('div', class_='genre')[0].find_all('div')[0]
            genres = ('|').join([genre.string for genre in genre_tags if not genre.string.isspace()])
        except:
            print "Genre Extraction Failure", current_url
            genres = "NA"

        # Extract the list of styles associated with the musician       
        try:
            style_tags = soup.find_all('div', class_='styles')[0].find_all('div')[0]
            styles = ('|').join([style.string for style in style_tags if not style.string.isspace()])
        except:
            print "Style Extraction Failure", current_url
            styles = "NA"

        # Extract the musician's influencers (musicians who were influenced him)
        # Only expand if the current artist belongs to the jazz genre
        try:
#             if "Jazz" in genres:
            influencers_list = soup.find_all('section', attrs={'class':'related influencers'})[0]\
            .find_next('ul')\
            .find_all('li')

            for influencer_item in influencers_list:
                influencer_name = influencer_item.text.strip()
                influencer_link = influencer_item.a.attrs['href']

                influence_relationship = [influencer_name, influencer_link, name, current_base_url]
                print influence_relationship
                influence_writer.writerow(influence_relationship)

                if influencer_link not in explored:
                    explored.add(influencer_link)
                    url_queue.append(influencer_link)
        except:
            print "Influencers Extraction Failure", current_url

        # Extract the musician's followers (musicians who were influenced by him)
        try:
#             if "Jazz" in genres:
            followers_list = soup.find_all('section', attrs={'class':'related followers'})[0]\
            .find_next('ul')\
            .find_all('li')

            for follower_item in followers_list:
                follower_name = follower_item.text.strip()
                follower_link = follower_item.a.attrs['href']

                follower_relationship = [name, current_base_url, follower_name, follower_link]
                print follower_relationship
                influence_writer.writerow(follower_relationship)

                if follower_link not in explored:
                    explored.add(follower_link)
                    url_queue.append(follower_link)
        except:
            print "Followers Extraction Failure", current_url
        
        influence_out.flush()


        # Write artist information
        try:
            artist_info = [name, current_base_url, active, genres, styles]
            print artist_info
            artist_writer.writerow(artist_info)
            artist_out.flush()
        except:
            print "Failure writing artist information", current_url

    except:
        # Sleep for just over an hour if blocked by server
        print "Request Failure: Sleeping for an hour..."
        sleep(60 * 61)

    

Size of queue: 1
Artists explored: 0
[u'Art Tatum', u'http://www.allmusic.com/artist/art-tatum-mn0000505770', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Buster Smith', u'http://www.allmusic.com/artist/buster-smith-mn0000643597', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Lester Young', u'http://www.allmusic.com/artist/lester-young-mn0000259529', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Ben Webster', u'http://www.allmusic.com/artist/ben-webster-mn0000793227', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083', u'Charlie Parker', 'http://www.allmusic.com/artist/charlie-parker-mn0000211758']
[u'Coleman Ha

Influencers Extraction Failure http://www.allmusic.com/artist/buster-smith-mn0000643597/related
Followers Extraction Failure http://www.allmusic.com/artist/buster-smith-mn0000643597/related
[u'Buster Smith', u'http://www.allmusic.com/artist/buster-smith-mn0000643597', u'1950s', u'Jazz|R&B;', u'Swing|Bop|Early R&B;']
Size of queue: 47
Artists explored: 49
[u'Fats Waller', u'http://www.allmusic.com/artist/fats-waller-mn0000162762', u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044']
[u'James P. Johnson', u'http://www.allmusic.com/artist/james-p-johnson-mn0000142860', u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044']
[u'Willie "The Lion" Smith', u'http://www.allmusic.com/artist/willie-the-lion-smith-mn0000687935', u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044']
[u'Bennie Moten', u'http://www.allmusic.com/artist/bennie-moten-mn0000129652', u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044'

[u'Bubber Miley', u'http://www.allmusic.com/artist/bubber-miley-mn0000528912', u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083']
[u'Louis Armstrong', u'http://www.allmusic.com/artist/louis-armstrong-mn0000234518', u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083']
[u'Fletcher Henderson', u'http://www.allmusic.com/artist/fletcher-henderson-mn0000152490', u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083']
[u'Frankie Trumbauer', u'http://www.allmusic.com/artist/frankie-trumbauer-mn0000168402', u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083']
[u'Sidney Bechet', u'http://www.allmusic.com/artist/sidney-bechet-mn0000033234', u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083']
[u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083', u'Art Pepper', u'http://www.allmusic.com/artist/art-pepper-mn0000505047']
[u'Benny Carter', u'http://www.a

[u'Earl Hines', u'http://www.allmusic.com/artist/earl-hines-mn0000455522', u'Jay McShann', u'http://www.allmusic.com/artist/jay-mcshann-mn0000225389']
[u'Bennie Moten', u'http://www.allmusic.com/artist/bennie-moten-mn0000129652', u'Jay McShann', u'http://www.allmusic.com/artist/jay-mcshann-mn0000225389']
[u'Count Basie', u'http://www.allmusic.com/artist/count-basie-mn0000127044', u'Jay McShann', u'http://www.allmusic.com/artist/jay-mcshann-mn0000225389']
Followers Extraction Failure http://www.allmusic.com/artist/jay-mcshann-mn0000225389/related
[u'Jay McShann', u'http://www.allmusic.com/artist/jay-mcshann-mn0000225389', u'1930s - 2000s', u'Jazz|Blues|Vocal', u'Big Band|Boogie-Woogie|Jazz Blues|Jump Blues|Piano Blues|Swing|Classic Female Blues|Urban Blues|Bop|Early Jazz|Jazz Instrument|Mainstream Jazz|Piano Jazz|Standards|Stride']
Size of queue: 117
Artists explored: 127
[u'Sidney Bechet', u'http://www.allmusic.com/artist/sidney-bechet-mn0000033234', u'Johnny Hodges', u'http://www.allm

[u'Benny Carter', u'http://www.allmusic.com/artist/benny-carter-mn0000790083', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Coleman Hawkins', u'http://www.allmusic.com/artist/coleman-hawkins-mn0000776363', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Henry "Red" Allen', u'http://www.allmusic.com/artist/henry-red-allen-mn0000675395', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Jabbo Smith', u'http://www.allmusic.com/artist/jabbo-smith-mn0000103408', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Barney Bigard', u'http://www.allmusic.com/artist/barney-bigard-mn0000785522', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Cozy Cole', u'http://www.allmusic.com/artist/cozy-cole-mn0000121483', u'Roy Eldridge', u'http://www.allmusic.com/artist/roy-eldridge-mn0000346637']
[u'Duke Ellington', u'http://www.allmusic.com/arti

...

[u'Steve Lukather', u'http://www.allmusic.com/artist/steve-lukather-mn0000042513', u'1980s - 2010s', u'Pop/Rock|R&B;', u'Alternative Pop/Rock|Alternative/Indie Rock|Blues-Rock|Contemporary Pop/Rock|Hard Rock']
Size of queue: 72
Artists explored: 2846
[u'Toto', u'http://www.allmusic.com/artist/toto-mn0000006320', u'1970s - 2010s', u'Pop/Rock', u'Contemporary Pop/Rock|Soft Rock|Album Rock|Film Score|Adult Contemporary']
Size of queue: 71
Artists explored: 2846
[u'Al Jolson', u'http://www.allmusic.com/artist/al-jolson-mn0000609215', u'Frankie Laine', u'http://www.allmusic.com/artist/frankie-laine-mn0000142178']
Followers Extraction Failure http://www.allmusic.com/artist/frankie-laine-mn0000142178/related
[u'Frankie Laine', u'http://www.allmusic.com/artist/frankie-laine-mn0000142178', u'1930s - 2000s', u'Vocal|Jazz', u'Traditional Pop|Vocal Pop|Dance Bands|Swing|Vocal Jazz']
Size of queue: 70
Artists explored: 2846
Style Extraction Failure http://www.allmusic.com/artist/irene-dunne-mn00001

Influencers Extraction Failure http://www.allmusic.com/artist/john-dankworth-mn0000065706/related
Followers Extraction Failure http://www.allmusic.com/artist/john-dankworth-mn0000065706/related
[u'John Dankworth', u'http://www.allmusic.com/artist/john-dankworth-mn0000065706', u'1940s - 2000s', u'Jazz|Classical', u'Big Band|Cool|Orchestral|Vocal Music']
Size of queue: 61
Artists explored: 2848
[u'Ben Webster', u'http://www.allmusic.com/artist/ben-webster-mn0000793227', u'Tony Coe', u'http://www.allmusic.com/artist/tony-coe-mn0000744111']
[u'Barney Bigard', u'http://www.allmusic.com/artist/barney-bigard-mn0000785522', u'Tony Coe', u'http://www.allmusic.com/artist/tony-coe-mn0000744111']
[u'Coleman Hawkins', u'http://www.allmusic.com/artist/coleman-hawkins-mn0000776363', u'Tony Coe', u'http://www.allmusic.com/artist/tony-coe-mn0000744111']
[u'Paul Gonsalves', u'http://www.allmusic.com/artist/paul-gonsalves-mn0000016925', u'Tony Coe', u'http://www.allmusic.com/artist/tony-coe-mn0000744111'

[u'Armando Peraza', u'http://www.allmusic.com/artist/armando-peraza-mn0000928568', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
[u'Art Blakey', u'http://www.allmusic.com/artist/art-blakey-mn0000928942', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
[u'Candido', u'http://www.allmusic.com/artist/candido-mn0000543045', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
[u'Chano Pozo', u'http://www.allmusic.com/artist/chano-pozo-mn0000166796', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
[u'Chico Hamilton', u'http://www.allmusic.com/artist/chico-hamilton-mn0000776301', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
[u'Jack Costanzo', u'http://www.allmusic.com/artist/jack-costanzo-mn0000123904', u'Jadir de Castro', u'http://www.allmusic.com/artist/jadir-de-castro-mn0001413399']
Followers Extraction Failure h

[u'The Yardbirds', u'http://www.allmusic.com/artist/the-yardbirds-mn0000489303', u'1960s - 2010s', u'Pop/Rock|Blues', u'Blues-Rock|British Blues|British Invasion|British Psychedelia|Psychedelic/Garage|Regional Blues|Rock & Roll']
Size of queue: 28
Artists explored: 2854
Active Period Extraction Failure http://www.allmusic.com/artist/caridad-cuervo-mn0000172327/related
[u'Caridad Cuervo', u'http://www.allmusic.com/artist/caridad-cuervo-mn0000172327', 'NA', u'Latin', u'Afro-Cuban|Tropical|Cuban Traditions']
Size of queue: 27
Artists explored: 2854
[u'Albita', u'http://www.allmusic.com/artist/albita-mn0000610889', u'1980s - 2000s', u'Latin', u'Latin Pop|Salsa|Tropical']
Size of queue: 26
Artists explored: 2854
[u'Angela Carrasco', u'http://www.allmusic.com/artist/angela-carrasco-mn0000044998', u'1990s', u'Latin', u'Latin Pop']
Size of queue: 25
Artists explored: 2854
[u'Bamboleo', u'http://www.allmusic.com/artist/bamboleo-mn0000135993', u'1990s - 2000s', u'Latin', u'Latin Pop|Salsa|Cuban 

Influencers Extraction Failure http://www.allmusic.com/artist/jack-costanzo-mn0000123904/related
Followers Extraction Failure http://www.allmusic.com/artist/jack-costanzo-mn0000123904/related
[u'Jack Costanzo', u'http://www.allmusic.com/artist/jack-costanzo-mn0000123904', u'1940s - 2000s', u'Jazz|Easy Listening|Latin', u'Latin Folk|Latin Jazz|Lounge|World Fusion']
